In [1]:
import time
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"

In [ ]:
colleges = [
    {
        "name": "Northeastern",
        "categories": [
            "https://news.northeastern.edu/category/university-news/",
            "https://news.northeastern.edu/category/world-news/",
            "https://news.northeastern.edu/category/science-technology/",
            "https://news.northeastern.edu/category/arts-entertainment/",
            "https://news.northeastern.edu/category/business/",
            "https://news.northeastern.edu/category/health/",
            "https://news.northeastern.edu/category/law/",
            "https://news.northeastern.edu/category/lifestyle/",
            "https://news.northeastern.edu/category/society-culture/",
            "https://news.northeastern.edu/category/sports/",
        ],
        "soup_selector": ".results-list .results-list__grid .post-link > a",
    },
    {
        "name": "Princeton",
        "categories": ["https://www.princeton.edu/news"],
        "soup_selector": "div.news-run.item > div > div > a",
    },
    {
        "name": "MIT",
        "categories": [
            "https://news.mit.edu/school/engineering",
            "https://news.mit.edu/school/humanities-arts-and-social-sciences",
            "https://news.mit.edu/school/architecture-and-planning",
            "https://news.mit.edu/school/management",
            "https://news.mit.edu/school/science",
            "https://news.mit.edu/school/mit-schwarzman-college-computing",
        ],
        "soup_selector": "a.term-page--news-article--item--title--link",
    },
    {
        "name": "Harvard",
        "categories": ["https://news.harvard.edu/gazette/section/news_plus/"],
        "soup_selector": "h2.hentry__title.wp-block-post-title > a",
    },
    {
        "name": "Chicago",
        "categories": ["https://news.uchicago.edu/lateststories"],
        "soup_selector": "a.c-list__item-titlelink",
    },
    {
        "name": "JHU",
        "categories": ["https://hub.jhu.edu/university-news/"],
        "soup_selector": "div.count-10 > div.force > div.text > div.accent-element > h5 > a",
    },
    {
        "name": "UPenn",
        "categories": ["https://penntoday.upenn.edu/news/archives"],
        "soup_selector": "a.overlink",
    },
    {
        "name": "Caltech",
        "categories": ["https://www.caltech.edu/about/news"],
        "soup_selector": ".block-NewsArticleListBlock .news-article-list .article-teaser__image-container a",
    },
    {
        "name": "Duke",
        "categories": [
            "https://today.duke.edu/arts-humanities",
            "https://today.duke.edu/business-economics",
            "https://today.duke.edu/campus-community",
            "https://today.duke.edu/environment-sustainability",
            "https://today.duke.edu/global",
            "https://today.duke.edu/health-medicine",
            "https://today.duke.edu/science-technology",
        ],
        "soup_selector": "div.paragraph--type--recent-stories-feed div.text-container > a",
    },
    {
        "name": "Northwestern",
        "categories": ["https://news.northwestern.edu/stories/"],
        "soup_selector": "h3 > a",
    },
    {
        "name": "Dartmouth",
        "categories": [
            "https://home.dartmouth.edu/news?content-types=article#articles-list"
        ],
        "soup_selector": ".views-row article a",
    },
    {
        "name": "Brown",
        "categories": ["https://www.brown.edu/news/all"],
        "soup_selector": ".news-items .news .component_body h3 a",
    },
    {
        "name": "WUSTL",
        "categories": ["https://source.wustl.edu/channels/newsroom/"],
        "soup_selector": "a.link-animate",
    },
    {
        "name": "Cornell",
        "categories": ["https://news.cornell.edu/archive"],
        "soup_selector": "h2 > a",
    },
    {
        "name": "NotreDame",
        "categories": ["https://news.nd.edu/latest-news/"],
        "soup_selector": "a.card-link",
    },
    {
        "name": "UCB",
        "categories": ["https://news.berkeley.edu/all-news/"],
        "soup_selector": "article.preview--post .preview__body h2 a",
    },
    {
        "name": "UCLA",
        "categories": ["https://newsroom.ucla.edu/stories"],
        "soup_selector": "h3.article-list-title > a",
    },
    {
        "name": "CMU",
        "categories": ["https://www.cmu.edu/news/stories"],
        "soup_selector": "a.link-box",
    },
    {
        "name": "Emory",
        "categories": ["https://news.emory.edu/tags/topic/news_releases/index.html"],
        "soup_selector": "li.tag-list-item > a",
    },
    {
        "name": "Georgetown",
        "categories": ["https://www.georgetown.edu/news/"],
        "soup_selector": ".c--articles h3 a",
    },
    {
        "name": "UMich",
        "categories": ["https://news.umich.edu/all-stories/"],
        "soup_selector": "li.clearfix > a",
    },
    {
        "name": "USC",
        "categories": ["https://today.usc.edu/tag/usc-news/"],
        "soup_selector": "article h3>a",
    },
    {
        "name": "Virginia",
        "categories": ["https://news.virginia.edu/content/all-news"],
        "soup_selector": "div.uva-today-news-item-title > a",
    },
    {
        "name": "UFL",
        "categories": [
            "https://news.ufl.edu/news-categories/agriculture-and-environment/",
            "https://news.ufl.edu/news-categories/arts-and-media/",
            "https://news.ufl.edu/news-categories/business-and-economics/",
            "https://news.ufl.edu/news-categories/campus-and-community/",
            "https://news.ufl.edu/news-categories/education/",
            "https://news.ufl.edu/news-categories/government-and-society/",
            "https://news.ufl.edu/news-categories/health-and-science/",
        ],
        "soup_selector": ".news-landing-body div.item .card a",
    },
    {
        "name": "WFU",
        "categories": ["https://news.wfu.edu/archive/"],
        "soup_selector": "h2 > a",
    },
    {
        "name": "Tufts",
        "categories": [
            "https://now.tufts.edu/activism-social-justice",
            "https://now.tufts.edu/arts-humanities",
            "https://now.tufts.edu/business-economics",
            "https://now.tufts.edu/climate-sustainability",
            "https://now.tufts.edu/food-nutrition",
            "https://now.tufts.edu/global-affairs",
            "https://now.tufts.edu/health",
            "https://now.tufts.edu/animal-health-medicine",
            "https://now.tufts.edu/points-view",
            "https://now.tufts.edu/politics-voting",
            "https://now.tufts.edu/science-technology",
        ],
        "soup_selector": "h3.teaser__title > a",
    },
    {
        "name": "UCSB",
        "categories": ["https://news.ucsb.edu/all-news"],
        "soup_selector": "span.field-content > a",
    },
    {
        "name": "UCI",
        "categories": [
            "https://news.uci.edu/category/art-and-humanities/",
            "https://news.uci.edu/category/athletics/",
            "https://news.uci.edu/category/campus-life/",
            "https://news.uci.edu/category/health/",
            "https://news.uci.edu/category/science-and-tech/",
            "https://news.uci.edu/category/society-and-community/",
        ],
        "soup_selector": "h3.posts-grid-wide__post-title > a",
    },
    {
        "name": "UCSD",
        "categories": ["https://today.ucsd.edu/archives"],
        "soup_selector": "h3.h4 > a",
    },
    {
        "name": "BostonCollege",
        "categories": ["https://www.bc.edu/content/bc-web/sites/bc-news.html"],
        "soup_selector": ".bc-news-article-list-2022 .article-block>a",
    },
    {
        "name": "Rochester",
        "categories": [
            "https://www.rochester.edu/newscenter/category/news/sci-tech/",
            "https://www.rochester.edu/newscenter/category/news/society-culture/",
            "https://www.rochester.edu/newscenter/category/news/university-news/",
        ],
        "soup_selector": ".loop-item__title>a",
    },
    {
        "name": "UCDavis",
        "categories": ["https://www.ucdavis.edu/news/latest"],
        "soup_selector": "h3.vm-teaser__title > a",
    },
    {
        "name": "UTexas",
        "categories": ["https://news.utexas.edu/archive/"],
        "soup_selector": "h2.ut-h3 > a",
    },
    {
        "name": "WISC",
        "categories": ["https://news.wisc.edu/latest-news/"],
        "soup_selector": "h2.headline > a",
    },
    {
        "name": "BostonU",
        "categories": ["https://www.bu.edu/today/browse/"],
        "soup_selector": "a.wp-prepress-component-story-card-link",
    },
    {
        "name": "UIUC",
        "categories": [
            "https://news.illinois.edu/category/research-news/",
            "https://news.illinois.edu/category/campus-news/",
        ],
        "soup_selector": ".post-card > .post-card-text > h2 > a",
    },
    {
        "name": "WilliamMary",
        "categories": ["https://news.wm.edu/latest-news/"],
        "soup_selector": "#main article h2 a",
    },
    {
        "name": "Brandeis",
        "categories": ["https://www.brandeis.edu/stories/2024/"],
        "soup_selector": ".wrap.wrap--xslim .feed .h6 a",
    },
    {
        "name": "Case",
        "categories": [
            "https://thedaily.case.edu/category/campus-community/",
            "https://thedaily.case.edu/category/university-news/",
            "https://thedaily.case.edu/category/research/",
            "https://thedaily.case.edu/category/people/",
        ],
        "soup_selector": "h2.title > a",
    },
    {
        "name": "Rutgers",
        "categories": ["https://www.rutgers.edu/news/all"],
        "soup_selector": "article.cc--news-card  h3 > a",
    },
    {
        "name": "Washington",
        "categories": ["https://www.washington.edu/news/the-latest-news-from-the-uw/"],
        "soup_selector": "h2 > a",
    },
    {
        "name": "Purdue",
        "categories": ["https://www.purdue.edu/newsroom/articles/"],
        "soup_selector": "a.purdue-home-cta-card--story",
    },
    {
        "name": "UMaryland",
        "categories": ["https://today.umd.edu/articles-list"],
        "soup_selector": "h2 > a",
    },
    {
        "name": "TAMU",
        "categories": ["https://today.tamu.edu/archives/"],
        "soup_selector": "div.heading-group > h2 > a",
    },
    {
        "name": "VT",
        "categories": ["https://news.vt.edu/articles.html"],
        "soup_selector": "ul.vt-list-items li .vt-list-item-row .vt-list-title a",
    },
    {
        "name": "Lehigh",
        "categories": ["https://news.lehigh.edu/all-stories"],
        "soup_selector": "div.col-md-8 h2 > a",
    },
    {
        "name": "OhioState",
        "categories": ["https://news.osu.edu/?h=1&t=News,Research%20News"],
        "soup_selector": "h3.pp_bigheadlines_heading>a",
    },
    {
        "name": "UofGeorgia",
        "categories": ["https://news.uga.edu/2025/"],
        "soup_selector": ".entry-header h2.entry-title>a",
    },
    {
        "name": "UofConnecticut",
        "categories": ["https://today.uconn.edu/archives/"],
        "soup_selector": "ul.archive-list li div.uct-small-story-row-headline>a",
    },
    {
        "name": "MichiganState",
        "categories": ["https://msutoday.msu.edu/view-all"],
        "soup_selector": "div.filtered-content div.row div.col-md-4 a",
    },
    {
        "name": "PennState",
        "categories": ["https://www.psu.edu/news/latest-news/"],
        "soup_selector": "article a#latest-news",
    },
    {
        "name": "NorthCarolinaState",
        "categories": ["https://news.ncsu.edu/2025/"],
        "soup_selector": "div.index-listing>a",
    },
    {
        "name": "SyracuseU",
        "categories": ["https://news.syr.edu/sections/"],
        "soup_selector": "div.article-list div.post h2.article-title a",
    },
    {
        "name": "UofMiami",
        "categories": ["https://news.miami.edu/"],
        "soup_selector": "div.news_container h2.stagsansmedium>a",
    },
    {
        "name": "TempleU",
        "categories": ["https://news.temple.edu/news/archives"],
        "soup_selector": "div.view-content div.views-row div.views-field-title a",
    },
    {
        "name": "ClemsonU",
        "categories": ["https://news.clemson.edu/topics/all-stories/"],
        "soup_selector": "#main article>a",
    },
    {
        "name": "FordhamU",
        "categories": ["https://now.fordham.edu/2025/"],
        "soup_selector": "article .content h2.post-title>a",
    },
    {
        "name": "Vanderbilt",
        "categories": ["https://news.vanderbilt.edu/2025/"],
        "soup_selector": ".article-page__body ul.story-list__list li>a",
    },
    {
        "name": "NYU",
        "categories": ["https://www.nyu.edu/about/news-publications/news/search.html"],
        "soup_selector": ".search-main .search-content .search-component .search-results-container .result-item.news .story-info>a",
    },
    {
        "name": "FSU",
        "categories": ["https://news.fsu.edu/category/news/"],
        "soup_selector": "main#content article div.row h2>a",
    },
    {
        "name": "GeorgeWashingtonU",
        "categories": ["https://gwtoday.gwu.edu/archives"],
        "soup_selector": "div.content main article div.view-content div.views-row p.gw-view-article-title>a",
    },
]

In [6]:
for index, college in enumerate(colleges):
    name = college["name"]
    categories = college["categories"]
    parse_str = college["soup_selector"]
    test = []
    for category in categories:
        response = requests.get(category, headers={"User-agent": user_agent})
        html = response.content
        soup = BeautifulSoup(html, "html.parser")
        links = soup.select(parse_str)
        links_array = [urljoin(category, link.get("href")) for link in links]
        if len(links_array) == 0:
            print(f"No links found for {name} in category {category}")
            break
        test.extend(links_array)
        time.sleep(3)
    print(f"#{index} Found {len(test)} links for {name}")

#0 Found 120 links for Northeastern
#1 Found 30 links for Princeton
#2 Found 90 links for MIT
#3 Found 30 links for Harvard
#4 Found 10 links for Chicago
#5 Found 10 links for JHU
#6 Found 10 links for UPenn
#7 Found 12 links for Caltech
#8 Found 112 links for Duke
#9 Found 10 links for Northwestern
#10 Found 9 links for Dartmouth
#11 Found 50 links for Brown
#12 Found 10 links for WUSTL
#13 Found 12 links for Cornell
#14 Found 24 links for NotreDame
#15 Found 16 links for UCB
#16 Found 25 links for UCLA
#17 Found 12 links for CMU
#18 Found 100 links for Emory
#19 Found 10 links for Georgetown
#20 Found 20 links for UMich
#21 Found 10 links for USC
#22 Found 8 links for Virginia
#23 Found 1495 links for UFL
#24 Found 7 links for WFU
#25 Found 110 links for Tufts
#26 Found 20 links for UCSB
#27 Found 60 links for UCI
#28 Found 10 links for UCSD
#29 Found 9 links for BostonCollege
#30 Found 30 links for Rochester
#31 Found 20 links for UCDavis
#32 Found 20 links for UTexas
#33 Found 20 l